OpenAI의 모델들의 이름들과 가격은 아래의 주소에서 확인하세요.

https://openai.com/pricing

# 1. 셋팅하기

코랩에서 OpenAI ChatGPT API를 실습하려면 openai 라이브러리를 설치해야 합니다. 다음 명령어를 입력합니다

```
[참고사항]

httpx==0.27.2를 추가 설치해주는 이유는 OpenAI 패키지와 httpx 패키지가 버전에 따라서 충돌이 발생하는 경우가 있기 때문입니다.

`TypeError: Client.__init__() got an unexpected keyword argument 'proxies'`

이슈: https://community.openai.com/t/error-with-openai-1-56-0-client-init-got-an-unexpected-keyword-argument-proxies/1040332/28
```

In [1]:
!pip install openai httpx==0.27.2


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


마지막 부분과 같이 ‘Successfully installed openai’라는
문구를 확인했다면 import openai를 통해 라이브러리를 불러옵니다. 그
후 openai.api_key의 값으로 OpenAI API 키 발급하는 사이트(https://platform.openai.com/account/api-keys) 에서 얻은 키
값을 입력합니다.

## 덮기

In [2]:
import openai

client = openai.OpenAI(api_key="")

사용량 확인하는 방법

https://platform.openai.com/account/usage

# 2. 답변 얻기

본격적으로 ChatGPT API를 사용해봅시다. 사용하는 코드의 형식은 다음
과 같습니다. model의 값으로 작성한 gpt-4o는 OpenAI의 API를
사용할 때 실행되는 ChatGPT의 다른 이름입니다. ChatGPT를 사용하기 위
해서는 모델의 값으로 gpt-4o 값을 사용해야 합니다. ChatGPT에게
작성할 프롬프트는 content의 값으로 입력하며, 이때 큰따옴표(")로 앞뒤를
감싸야 합니다.

```
completion = client.chat.completions.create(
  model="<사용하고자 하는 모델명>",
  messages=[
    {"role": "system", "content": "<정해주고 싶은 역할>"},
    {"role": "user", "content": "<작성하고 싶은 Prompt>"}
  ]
)
```

API를 활용해 ChatGPT에게 “2020년 월드시리즈에서 누가 우승했어?”라
고 물어보겠습니다.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {"role": "user", "content": "2020년 월드시리즈에서는 누가 우승했어?"}
    ]
)

In [ ]:
# 답변을 받는 방법
print(response.choices[0].message.content)

2020년 월드시리즈에서는 로스앤젤레스 다저스가 우승했습니다. 다저스는 탬파베이 레이스를 상대로 6경기 만에 시리즈를 승리로 마무리 지었습니다. 이로써 다저스는 32년 만에 월드시리즈 타이틀을 확보하게 되었습니다.


# 3. 역할 부여해주기

ChatGPT API를 통해 ChatGPT를 사용할 때는 ChatGPT에게 역할
을 지시할 수 있습니다. 여기서 역할 지시란 ChatGPT가 앞으로 답변할 때
ChatGPT가 해당 역할로서 답변을 하라는 의미입니다. 앞으로 안내하는 역
할 지시 방법은 항상 지시한 역할대로 동작한다는 보장은 없지만 역할 지시문
에 따라 답변 자체의 방향성을 바꿔버리기도 합니다.  

역할을 지시하려면 기존 코드에서 messages=[] 안에 {"role": "system",
"content": "<ChatGPT가 수행할 역할>"}를 추가로 작성합니다. 예를 들
어, ChatGPT에게 친절하게 답변해주는 비서라는 역할을 부여해봅시다. 코드
는 다음과 같습니다.  

이와 같이 거대 언어 모델의 전체적인 방향성을 지시하는 입력을 `시스템 프롬프트`라고 합니다.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "너는 친절하게 답변해주는 비서야. 답변은 반말로 해줘"},
      {"role": "user", "content": "2020년 월드시리즈에서는 누가 우승했어?"}
    ]
)

In [ ]:
# 답변을 받는 방법
print(response.choices[0].message.content)

2020년 월드시리즈에서는 로스앤젤레스 다저스가 우승했어.


동일하게 로스앤젤레스 다저스가 우승했다는 답변을 하지만 답변이 달라집니다.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "You must only answer users' questions in English. This must be honored. You must only answer in English."},
      {"role": "user", "content": "2020년 월드시리즈에서는 누가 우승했어?"}
    ]
)

In [ ]:
print(response.choices[0].message.content)

The Los Angeles Dodgers won the 2020 World Series.


이번에는 입력을 번역하라고 해봅시다.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "You are a Korean-English translator. Translate the user's input into English."},
      {"role": "user", "content": "2020년 월드시리즈에서는 누가 우승했어?"}
    ]
)

In [ ]:
print(response.choices[0].message.content)

The Los Angeles Dodgers won the 2020 World Series.


ChatGPT가 사용자의 질문에 대답하는 것이 아니라 질문을 영어로 번역하여 답변합니다.

이러한 시스템 프롬프트는 웹 사이트에서 사용하는 GPT-4에도 이미 적용되어져 있습니다.  
다만 UI 상으로 보여지지 않을 뿐입니다.  

GPT-4 시스템 프롬프트 유출: https://news.hada.io/topic?id=13326

또한 system, user, assisstant 형태로 데이터를 처리하는 것은 GPT-4 뿐만이 아닙니다.  

Llama-3, Qwen2 자료: https://colab.research.google.com/drive/1qF3ZiwOe6SS_8JnHoPl4iZY3eNZGTpGW?usp=sharing#scrollTo=GTMCZNFmfm8q

# 4. 이전 대화를 포함하여 답변하기

ChatGPT는 답변할 때 이전 질문과 답변을 모두 고려하여 답변하는 특징이 있습니다. ChatGPT API를 이용하면 ChatGPT에게 답변을 요청할 때 ChatGPT에게 ‘앞서 네가 이런 답변을 한 상태였다’라는 정보를 주는 것이 가능합니다. 이것은 사용자가 가정하는 것이지만 ChatGPT는 마치 과거에 ChatGPT 자신이 답변한 것으로 가정하고, 추가
답변을 합니다.  

이렇게 질문을 작성하려면 messages=[ ] 안에   
`{"role": "user", "content": "<사용자의 이전 프롬프트>"}`를 작성한 후   
`{"role": "assistant", "content": "<ChatGPT의 이전 답변>"}`을 추가로 작성하고,  
다시 `{"role": "user", "content": "<ChatGPT 이전 답변 이후의 사용자의 프롬프트>"}`를 번갈아 작성하면 됩니다.  

실제 코드를 통해 확인해봅시다.  

다음 코드에서는 사용자가 ChatGPT에게 “2002년 월드컵에서 가장 화제
가 되었던 나라는 어디야?”라고 질문한 후,  
ChatGPT가 “바로 예상을 뚫고 4강 진출 신화를 일으킨 한국입니다.”라고 답변했다고 가정하고, 사용자가 다시 “그 나라가 화제가 되었던 이유를 자세하게 설명해줘”라고 질문한 상태를 가정하고 ChatGPT의 답변을 얻습니다.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {"role": "user", "content": "2002년 월드컵에서 가장 화제가 되었던 나라는 어디야?"},
        {"role": "assistant", "content": "바로 예상을 뚫고 4강 진출 신화를 일으킨 한국입니다."},
        {"role": "user", "content": "그 나라가 화제가 되었던 이유를 자세하게 설명해줘"}
    ]
)

In [ ]:
print(response.choices[0].message.content)

2002년 월드컵에서 대한민국은 여러 이유로 큰 화제가 되었습니다. 주최국 중 하나였던 대한민국은 당시 월드컵 역사상 가장 성공적인 성과를 거두며 많은 주목을 받았습니다. 다음은 대한민국이 화제가 된 주요 이유들입니다:

1. **역대 최고 성적**: 대한민국은 아시아 국가로서는 처음으로 월드컵 4강에 진출하는 쾌거를 이뤘습니다. 이는 아시아 축구 역사상 최고의 성과 중 하나로 기록되었으며, 대한민국은 이를 통해 세계 축구 무대에서 주목받았습니다.

2. **히딩크 매직**: 당시 대한민국의 감독이었던 거스 히딩크는 팀을 효과적으로 이끌어 놀라운 전략과 전술로 주목받았습니다. 그의 리더십은 팀을 결속시키고 선수들의 잠재력을 최대한 발휘하게 했습니다.

3. **강호들을 꺾은 승리**: 대한민국은 대회 진행 중 이탈리아와 스페인 같은 축구 강호들을 꺾으며 큰 화제를 모았습니다. 특히 16강전에서 이탈리아를, 8강전에서 스페인을 꺾는 과정은 많은 팬들에게 깊은 인상을 남겼습니다.

4. **열정적인 홈 팬들의 응원**: 월드컵이 한국에서 개최된 만큼, 한국 팬들의 열정적인 응원과 서포터즈의 활동도 크게 주목받았습니다. 붉은악마로 알려진 대한민국 서포터즈는 각 경기를 뜨겁게 응원하며 세계적으로 인기를 끌었습니다.

5. **특별한 순간들**: 특히 안정환의 골든골과 박지성의 인상적인 플레이 등은 많은 축구 팬들에게 강렬한 인상을 남겼습니다.

이러한 요소들이 결합되어 2002년 월드컵에서 대한민국은 세계 축구 무대에서 큰 화제를 불러일으켰습니다.


# 5. 너무나 쉽게 AI 서비스 개발해보기

## 5-1. 뉴스 기사를 분석해주는 인공지능

앞으로 AI 서비스를 개발할 때는 정해주고 싶은 역할에 ChatGPT가 수행할 작업을 구체적으로 명시해주면 됩니다.

```
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {"role": "system", "content": "<정해주고 싶은 역할>"},
        {"role": "user", "content": "<작성하고 싶은 Prompt>"}
    ]
)
```

In [ ]:
def return_answer(input_text=''):
    system_prompt = """특정 문서가 입력되면 다음과 같은 형태로 문서를 분석하십시오.

1. 주어진 입력:에 대해서 반드시 주제:, 요약:, 가능한 질문: 이 세가지를 순차적으로 작성해야 합니다.
2. 주제:는 입력 문서의 주제를 한 줄로 요약합니다.
3. 요약:은 입력 문서를 5줄로 요약합니다.
4. 가능한 질문:은 입력 문서로부부터 사람들이 할 수 있는 질문 세 가지를 파이썬 리스트 형태로 작성합니다.
5. 가능한 질문:이 반드시 ["질문1", "질문2", "질문3"]과 같이 파이썬 리스트 형태로 작성되어야 하는 점에 유의하십시오.

이제 시작합니다."""

    user_content = "입력: " + input_text + '\n주제:'

    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_content}
      ]
    )
    return response.choices[0].message.content

In [ ]:
test = '''새처럼 움직이는 항공 로봇 '레이븐'.

스위스 로잔 연방 공대(EPFL) 신원동 박사팀이 개발했는데요.

연구팀은 새의 엉덩이와 발목, 발을 참고해 뒷다리를 만들었습니다.

실제 새처럼 땅에서 걷고, 틈새를 건너뛰고, 위로 뛰어오릅니다.

껑충 뛰면서 날아오르는 이 기능이 핵심인데요.

기존 고정익 항공기처럼 지상을 달리지 않고 제자리에서 날아오릅니다.

연구팀은 레이븐의 점프 이륙이 점프하지 않고 날아오를 때보다 에너지 효율이 더 높다고 설명했는데요.

또 기존 고정익 항공기에 이 기능을 도입하면 복잡한 지형에서도 효율적으로 쓸 수 있을 거라고 덧붙였습니다.

#새 #로봇 #항공 #공대 #스위스 #신원동 #박사 #점프'''

In [ ]:
result = return_answer(input_text=test)
print(result)

주제: 새처럼 움직이는 점프 이륙 기능을 가진 항공 로봇 '레이븐'의 개발

요약: 스위스 로잔 연방 공대의 신원동 박사팀이 새의 움직임을 모방한 항공 로봇 '레이븐'을 개발했습니다. 레이븐은 새의 엉덩이, 발목, 발 구조를 참고하여 뒷다리를 설계하여 실제 새처럼 걷고 뛰는 동작을 수행할 수 있습니다. 특히 이 로봇은 고정익 항공기처럼 긴 활주 없이 제자리에서 껑충 뛰어오르며 날아오르는 기능이 핵심입니다. 연구에 따르면, 이러한 점프 이륙은 에너지 효율 면에서 이점이 있으며, 복잡한 지형에서도 효율적으로 작동할 수 있습니다. 연구팀은 이 기술을 기존 항공기에도 적용할 수 있음을 강조했습니다.

가능한 질문: ["레이븐은 어떻게 새의 동작을 모방하였나요?", "레이븐의 점프 이륙 기능의 장점은 무엇인가요?", "이 연구가 항공기 설계에 어떤 영향을 미칠 수 있나요?"]


In [ ]:
test = '''
꿈의 에너지’라고 불리는 핵융합 에너지가 5년 뒤 현실이 될지도 모른다. 마이크로소프트가 5년 뒤부터 핵융합으로 만든 전기를 사서 쓰겠다는 전기공급 계약을 맺었다.

마이크로소프트에게 전기를 공급할 곳은 미국 스타트업 ‘헬리온 에너지’다. 5월 10일 핵융합 발전 스타트업 헬리온 에너지가 마이크로소프트와 계약을 맺었다고 밝혔다. 계약 조건은 2028년부터 헬리온 에너지가 핵융합 발전을 시작하고, 1년 뒤 50메가와트(MW) 이상을 발전해 마이크로소프트에 공급하는 것이다. 계약을 이행하지 못할 경우 헬리온 에너지가 마이크로소프트에 위약금을 내야 한다.

하지만 전문가들의 전망은 긍정적이지만은 않다. 사용할 수 있는 정도의 전기를 만들어 내는 실증 시험까지 거친 핵융합 연구가 아직 없기 때문이다. 대신 현실적인 목표 시기는 2040년 이후가 될 것으로 보인다.

핵융합 발전 연구는 크게 공공 개발과 민간 개발, 둘로 나뉘는데 둘 중 핵융합 발전 목표 시기가 조금 더 빠른 민간 개발의 목표 시기가 2030년경이다. 다만 지난해 11월 유럽 위원회가 낸 보고서에서는 이 시기를 시험 운영 목표로 하는 건 실패 위험이 높을 것으로 내다봤다. 보고서는 실패 위험이 중간 정도인 시기는 가장 빠른 것이 2036년, 가장 늦은 게 2050년이라고 분석했다.
'''

In [ ]:
result = return_answer(input_text=test)
print(result)

주제: 핵융합 에너지의 상용화 가능성과 이를 둘러싼 논란 및 전망.

요약: 
마이크로소프트는 2028년부터 핵융합 에너지를 사용하겠다는 계약을 미국 스타트업 헬리온 에너지와 체결했다. 계약에 따르면 헬리온 에너지는 2028년 핵융합 발전을 시작해, 2029년부터 50메가와트 이상의 전기를 공급해야 한다. 그러나 실증 시험을 거친 핵융합 발전 연구가 아직 부족해 전문가들은 2040년 이후가 실현 가능한 시기로 보고 있다. 공공 개발보다 약간 빠른 민간 개발의 목표 시기는 2030년대 초반으로 잡혀있지만, 유럽 위원회의 보고서는 이 시기가 실패 위험이 높다고 지적했다. 보고서에 따르면 가장 빠른 실현 가능 시기는 2036년, 가장 늦은 시기는 2050년으로 예상된다.

가능한 질문: ["마이크로소프트가 헬리온 에너지와 체결한 계약의 주요 조건은 무엇인가?", "전문가들이 핵융합 에너지의 상용화 시기를 회의적으로 보는 이유는 무엇인가?", "유럽 위원회의 보고서에 따르면 핵융합 에너지 상용화의 실현 가능 시기는 언제인가?"]


## 5-2. 키워드 추출

만약 프롬프트를 아무리 써도 원하는대로 답변이 나오지 않는다면 예시를 사용하세요.  

예시는 답변 방향을 컨트롤 할 수 있는 강력한 방법입니다.

```
You are an expert at extracting keywords from a given sentence. Extract keywords from a given sentence that are key to the context.

[Example]
input: '외신에 따르면 중국 이카이글로벌 보도를 인용해 모더나가 중국 전용 mRNA 백신 개발을 위해 중국에 최대 10억달러(약 1조3017억원)를 투자하기로 했다고 보도했다.'
keyword: ['모더나', 'mRNA', '1조3017억원']

Now the sentence you want to extract keywords from.
The output format must be a list in Python.

Input: str(input_text)
Keyword:
```

In [ ]:
def return_answer(input_text):
    system_prompt = """You are an expert at extracting keywords from a given sentence. Extract keywords from a given sentence that are key to the context.

    [Example]
    input: '외신에 따르면 중국 이카이글로벌 보도를 인용해 모더나가 중국 전용 mRNA 백신 개발을 위해 중국에 최대 10억달러(약 1조3017억원)를 투자하기로 했다고 보도했다.'
    keyword: ['모더나', 'mRNA', '1조3017억원']

    Now the sentence you want to extract keywords from.
    The output format must be a list in Python.
    """
    user_content = "input: " + input_text + "\nkeyword"

    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_content}
      ],
    temperature=0
    )
    return response.choices[0].message.content

In [ ]:
text = '''3일 업계에 따르면 서울 신라호텔의 더 파크뷰는 12월 1∼20일 저녁 가격을 19만5000원으로, 21∼31일 저녁 가격을 21만 5000원으로 각각 인상한다. 평소 평일·주말 저녁 가격이 18만5000원이던 것을 고려하면 각각 1만원과 3만원 올랐다. 가격 인상에도 불구하고 더 파크뷰의 12월 주말 예약은 대부분 찼고 12월 평일 예약 역시 빠르게 마감되고 있다고 신라호텔 측은 전했다.

롯데호텔 서울의 뷔페 라세느는 12월 평일·주말 저녁 가격을 19만원으로 기존 대비 1만원 올린다. 크리스마스 연휴 때인 23∼25일과 연말 30∼31일 저녁 가격은 20만5000원이다. 롯데호텔 서울의 라세느도 현재 크리스마스 이브인 12월 24일과 크리스마스 당일인 12월 25일 예약은 대부분 마감됐다.

워커힐 호텔앤리조트는 그랜드 워커힐에서 운영하던 더뷔페를 비스타 워커힐로 확장 이전해 이날 가격을 올렸다. 평일·주말 저녁 가격은 15만9000원에서 18만9000원으로 3만원 인상됐다. 다만 워커힐은 이번 가격 책정에 인플레이션(물가 상승)을 미리 반영한 만큼 연말을 앞두고 추가 인상을 단행하지 않을 계획이라고 밝혔다.

그랜드 인터컨티넨탈 서울 파르나스와 인터컨티넨탈 서울 코엑스도 다음주께 인상된 가격을 공지할 계획이다.'''

In [ ]:
return_answer(input_text=text)

"['서울 신라호텔', '더 파크뷰', '19만5000원', '21만5000원', '롯데호텔 서울', '라세느', '19만원', '20만5000원', '워커힐 호텔앤리조트', '더뷔페', '비스타 워커힐', '18만9000원', '인플레이션', '그랜드 인터컨티넨탈 서울 파르나스', '인터컨티넨탈 서울 코엑스']"

## 5-3. 그 외 간단한 예시들

few-shot prompting: 예시가 2개 이상인 경우를 의미한다.

In [ ]:
def return_label(text):
    prompt = '''When input comes in, you should return the main keywords that represent the topic of the content in the form of a Python list.
        Keywords should be in the form of a Python list of no more than four compound nouns that best reflect the subject.

        Example)
        input: 제주에서 출발하는 김포행 아시아나항공 여객기에 문제가 발생해 탑승까지 마친 승객 190여명의 발이 묶였다. 기내 탑승까지 마쳤던 승객들은 다시 비행기에서 내려 현재 제주공항에서 대기 중이다.
        13일 한국공항공사 제주공항과 아시아나항공 등에 따르면 이날 오전 6시 30분 제주발 김포행 아시아나 OZ8900편 항공기에서 비상 착수 장비인 슬라이드 고정 프레임에 문제가 발생했다. 아시아나항공 관계자는 “항공기 자체 결함은 아니다”라면서 “손님이 탑승 중 비상구 레버와 접촉해 비상구 사용 불가 상태가 되면서 손님을 분산해 출발할 예정”이라고 전했다.
        Keywords: ['김포행 아시아나항공 여객기', '승객 190여명', '비상 착수 장비', '슬라이드 고정 프레임에 문제']

        input: 서울 노원구에서 중고폰 할인매장을 운영하는 40대 정모씨는 중고거래 플랫폼 ‘당근마켓’ 알림을 켜놓고 중고폰을 매입하고 있다. 상태 좋은 중고폰이 많이 나오는 당근마켓 등으로 매입처를 확대한 것이다. 중고폰 중에서도 아이폰X 이후 모델은 매입 ‘1순위’다. 정씨는 “중고폰을 찾는 소비자들이 늘고 있지만 아이폰X 이후 모델은 없어서 못 팔 정도다”라며 “청소년과 대학생이 매장을 많이 찾는데 이들 중 90% 이상이 중고 아이폰을 구입해 간다”라고 했다.
        Keywords: ['당근마켓', '중고폰을 매입', '아이폰X', '청소년과 대학생']
        '''

    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt},
          {"role": "user", "content": "input: " + text + "\nkeywords: "}
      ]
    )
    return response.choices[0].message.content

In [ ]:
input_text = "13일 금융투자업계에 따르면 최근 중소형 증권사에서 경력이 길지 않은 스몰캡 애널리스트들이 연이어 퇴사한 것으로 알려졌다. 최근 A투자증권에 재직하던 스몰캡 담당 연구원은 독립리서치로 자리를 옮겼다. B증권에서는 종목 보고서 발간 기업이 거래정지돼 도마 위에 올랐던 한 연구원이 올해 초 회사를 떠났다. 리서치 강자로 꼽히는 C증권에서는 리서치 보조(RA)만 하다가 연구원으로 정식 데뷔를 못하고 떠난 사례가 여럿이다."
return_label(input_text)

"['중소형 증권사', '스몰캡 애널리스트 퇴사', '독립리서치', '리서치 보조']"

zero-shot prompting: 예시없이 프롬프트를 입력하는 경우.

In [ ]:
def return_label(text):
    prompt = '''주어진 텍스트로부터 적절한 제목을 만들어줘. 제목은 간결하고 너무 길어서는 안 돼.'''

    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt},
          {"role": "user", "content": text}
      ]
    )
    return response.choices[0].message.content

In [ ]:
input_text = '''지난 11일 국회 정무위원회 전체 회의에선 이복현 금융감독원장의 해외 출장길이 도마 에 올랐다. 이 원장이 지난 8일부터 닷새간 동남아 싱가포르·인도네시아·태국 등 주요 3개국 출장을 떠나 이날 회의에 참석하지 못했기 때문이다. 금감원에선 이준수·함용일 부원장이 전체 회의에 대참했다.

이 원장의 이번 출장은 국내 금융사의 투자 유치와 해외진출 확대를 지원하기 위해서다. 이 출장에는 윤종규 KB금융그룹 회장, 함영주 하나금융그룹 회장을 비롯해 최현만 미래에셋증권 회장, 정일문 한국투자증권 사장, 홍원학 삼성화재 사장, 원종규 코리안리 사장 등 6개 금융사 최고경영자(CEO)도 동행했다.

야당의 지적처럼 금융권 일각에선 이번 출장을 두고 다소 의아하다는 시각이 있었다. 금감원이 해외에서 IR 행사를 하긴 하지만, 금융위원장이 아니라 금감원장이 직접 나간 적은 드물어서다. 금감원도 이를 의식한 듯 관련 보도자료에 ‘이번 금감원장의 해외 IR 참여는 몇몇 금융사 회장들이 지난해부터 요청해서 이뤄진 것’이라는 해명을 덧붙이기도 했다.'''

return_label(input_text)

'금감원장 해외 출장 논란'

one-shot prompting: 예시가 1개인 경우를 의미.

In [ ]:
def return_label(text):
    prompt = '''주어진 텍스트가 긍정인지 중립인지 부정인지 예측하시오. 당신의 답변은 오로지 [positive, negative, neutral] 셋 중 하나여야만 합니다.
    ex) 'SK하이닉스가 2분기 실적에서 역대 최고를 기록하였다' => "positive"'''

    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt},
          {"role": "user", "content": text + " => "}
      ]
    )
    return response.choices[0].message.content

In [ ]:
return_label('씨씨에스 로고 씨씨에스 주가가 상한가다. 3일 기준 씨씨에스는 29.84%(825원) 상승한 3590원에 거래를 마쳤다. 상온 초전도체 이슈가 부각되면서 주가가 연일 영향을 받는 모양새다.')

'positive'

# 6. 업무자동화 (실제 사례)

In [ ]:
import pandas as pd
import ast

# 데이터를 리스트의 형태로 정리
data = {
    "수리일자": ["2023-09-07", "2021-10-08", "2022-02-23", "2022-04-06", "2021-02-20", "2021-02-26", "2021-04-13", "2021-07-30", "2022-07-12", "2022-07-13", "2022-07-18", "2022-07-18", "2022-08-11", "2022-09-13", "2022-10-04", "2022-10-26", "2022-11-11", "2022-11-14"],
    "고장내용": [
        "출하장 입구 에어 50A 유니온 에어리크 재조임",
        "SHIELD ROOM 화풍기 철거부위 마감,판넬벽체문타공",
        "F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정",
        "와이어 전진동작이상 점검 중 정상 작동됨",
        "클램프기동이상 오일보충 에어 제거",
        "피이더 조작판 매입 콘센트 고정",
        "QDC 파손 클램프 교체",
        "광세샤 고정블록 볼트 마모 600ton 것으로 대체",
        "휠 회전 작동불량 점검 세기정공 AS 요청",
        "클러치 ASS'Y 작동이상 분해출문(세기정공)",
        "클러치 assy 불량으로 전자접촉기교체",
        "클러치 밋션 기어오일보출 ATF-3",
        "휨 가지센서 단선 교체",
        "안전1행정 타발정지위치이상 리미트캠스위치교체및조정,상사점리미트스위치교체",
        "QDC 클램프 기동이상 에어빼기",
        "피이더 소재 진입이상 캠 스위치 조정",
        "카운터 완료시 정지시점 불량 카운타 각도 조정",
        "NC FEEDER 코일 이송거리 이상 재 셋팅"
    ],
    "고장부품": [None if x in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] else x for x in ["50A 유니온", "SHIELD ROOM 화풍기"] + [None]*16],
    "불량유형": [None if x in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] else x for x in ["에어리크", "철거부위 마감"] + [None]*16],
    "조치내용": [None if x in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] else x for x in ["재조임", "판넬벽체문타공"] + [None]*16]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

In [ ]:
df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2023-09-07,출하장 입구 에어 50A 유니온 에어리크 재조임,50A 유니온,에어리크,재조임
1,2021-10-08,"SHIELD ROOM 화풍기 철거부위 마감,판넬벽체문타공",SHIELD ROOM 화풍기,철거부위 마감,판넬벽체문타공
2,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,None,None,None
3,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,None,None,None
4,2021-02-20,클램프기동이상 오일보충 에어 제거,None,None,None
5,2021-02-26,피이더 조작판 매입 콘센트 고정,None,None,None
6,2021-04-13,QDC 파손 클램프 교체,None,None,None
7,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,None,None,None
8,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,None,None,None
9,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),None,None,None


In [ ]:
df[df['고장부품'].isnull()]

,수리일자,고장내용,고장부품,불량유형,조치내용
2,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,None,None,None
3,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,None,None,None
4,2021-02-20,클램프기동이상 오일보충 에어 제거,None,None,None
5,2021-02-26,피이더 조작판 매입 콘센트 고정,None,None,None
6,2021-04-13,QDC 파손 클램프 교체,None,None,None
7,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,None,None,None
8,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,None,None,None
9,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),None,None,None
10,2022-07-18,클러치 assy 불량으로 전자접촉기교체,None,None,None
11,2022-07-18,클러치 밋션 기어오일보출 ATF-3,None,None,None


In [ ]:
sub_df = df[df['고장부품'].isnull()]
sub_df.reset_index(drop=True, inplace=True)
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,None,None,None
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,None,None,None
2,2021-02-20,클램프기동이상 오일보충 에어 제거,None,None,None
3,2021-02-26,피이더 조작판 매입 콘센트 고정,None,None,None
4,2021-04-13,QDC 파손 클램프 교체,None,None,None
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,None,None,None
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,None,None,None
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),None,None,None
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,None,None,None
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,None,None,None


In [ ]:
inputs= sub_df['고장내용'].to_list()
print(inputs)

['F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정', '와이어 전진동작이상 점검 중 정상 작동됨', '클램프기동이상 오일보충 에어 제거', '피이더 조작판 매입 콘센트 고정', 'QDC 파손 클램프 교체', '광세샤 고정블록 볼트 마모 600ton 것으로 대체', '휠 회전 작동불량 점검 세기정공 AS 요청', "클러치 ASS'Y 작동이상 분해출문(세기정공)", '클러치 assy 불량으로 전자접촉기교체', '클러치 밋션 기어오일보출 ATF-3', '휨 가지센서 단선 교체', '안전1행정 타발정지위치이상 리미트캠스위치교체및조정,상사점리미트스위치교체', 'QDC 클램프 기동이상 에어빼기', '피이더 소재 진입이상 캠 스위치 조정', '카운터 완료시 정지시점 불량 카운타 각도 조정', 'NC FEEDER 코일 이송거리 이상 재 셋팅']


In [ ]:
def return_label(text):
    prompt = '''주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
    이때 파이썬의 Dictionary 형태로 반환하시오.

    ex)
    고장내용: '출하장 입구 에어 50A 유니온 에어리크 재조임'
    출력: {"고장부품": "50A 유니온", "불량유형" : "에어리크", "조치내용" : "재조임"}

    고장내용: 'SHIELD ROOM 화풍기 철거부위 마감,판넬벽체문타공	'
    출력: {"고장부품": "SHIELD ROOM 화풍기	", "불량유형" : "철거부위 마감", "조치내용" : "판넬벽체문타공"}'''

    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt},
          {"role": "user", "content": "고장내용: " + text + "\n출력: "}
      ],
    temperature=0
    )
    return response.choices[0].message.content

In [ ]:
sub_df['고장내용'].loc[0]

'F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정'

In [ ]:
return_label(sub_df['고장내용'].loc[0])

'{"고장부품": "F/P", "불량유형": "기능상실", "조치내용": "PLC 확인 및 클램프 감지센샤 조정"}'

In [ ]:
sub_df['고장내용'].loc[1]

'와이어 전진동작이상 점검 중 정상 작동됨'

In [ ]:
return_label(sub_df['고장내용'].loc[1])

'{"고장부품": "와이어", "불량유형": "전진동작이상", "조치내용": "점검 중 정상 작동됨"}'

In [ ]:
result = []
for input in sub_df['고장내용'].to_list():
  result.append(return_label(input))

In [ ]:
result

['{"고장부품": "F/P", "불량유형": "기능상실", "조치내용": "PLC 확인 및 클램프 감지센샤 조정"}',
 '{"고장부품": "와이어", "불량유형": "전진동작이상", "조치내용": "점검 중 정상 작동됨"}',
 '{"고장부품": "클램프", "불량유형": "기동이상", "조치내용": "오일보충 에어 제거"}',
 '{"고장부품": "피이더 조작판", "불량유형": "매입 콘센트", "조치내용": "고정"}',
 '{"고장부품": "QDC", "불량유형": "파손", "조치내용": "클램프 교체"}',
 '{"고장부품": "고정블록 볼트", "불량유형": "마모", "조치내용": "600ton 것으로 대체"}',
 '{"고장부품": "휠", "불량유형": "회전 작동불량", "조치내용": "점검 세기정공 AS 요청"}',
 '{"고장부품": "클러치 ASS\'Y", "불량유형": "작동이상", "조치내용": "분해출문(세기정공)"}',
 '{"고장부품": "클러치 assy", "불량유형": "불량", "조치내용": "전자접촉기교체"}',
 '{"고장부품": "클러치 밋션", "불량유형": "기어오일보출", "조치내용": "ATF-3"}',
 '{"고장부품": "가지센서", "불량유형": "단선", "조치내용": "교체"}',
 '{"고장부품": "안전1행정", "불량유형": "타발정지위치이상", "조치내용": "리미트캠스위치교체및조정, 상사점리미트스위치교체"}',
 '{"고장부품": "QDC 클램프", "불량유형": "기동이상", "조치내용": "에어빼기"}',
 '{"고장부품": "피이더 소재", "불량유형": "진입이상", "조치내용": "캠 스위치 조정"}',
 '{"고장부품": "카운터", "불량유형": "정지시점 불량", "조치내용": "각도 조정"}',
 '{"고장부품": "NC FEEDER 코일", "불량유형": "이송거리 이상", "조치내용": "재 셋팅"}']

In [ ]:
result[0]

'{"고장부품": "F/P", "불량유형": "기능상실", "조치내용": "PLC 확인 및 클램프 감지센샤 조정"}'

In [ ]:
print(type(result[0]))

<class 'str'>


In [ ]:
result = [ast.literal_eval(elem) for elem in result]
print(result)

[{'고장부품': 'F/P', '불량유형': '기능상실', '조치내용': 'PLC 확인 및 클램프 감지센샤 조정'}, {'고장부품': '와이어', '불량유형': '전진동작이상', '조치내용': '점검 중 정상 작동됨'}, {'고장부품': '클램프', '불량유형': '기동이상', '조치내용': '오일보충 에어 제거'}, {'고장부품': '피이더 조작판', '불량유형': '매입 콘센트', '조치내용': '고정'}, {'고장부품': 'QDC', '불량유형': '파손', '조치내용': '클램프 교체'}, {'고장부품': '고정블록 볼트', '불량유형': '마모', '조치내용': '600ton 것으로 대체'}, {'고장부품': '휠', '불량유형': '회전 작동불량', '조치내용': '점검 세기정공 AS 요청'}, {'고장부품': "클러치 ASS'Y", '불량유형': '작동이상', '조치내용': '분해출문(세기정공)'}, {'고장부품': '클러치 assy', '불량유형': '불량', '조치내용': '전자접촉기교체'}, {'고장부품': '클러치 밋션', '불량유형': '기어오일보출', '조치내용': 'ATF-3'}, {'고장부품': '가지센서', '불량유형': '단선', '조치내용': '교체'}, {'고장부품': '안전1행정', '불량유형': '타발정지위치이상', '조치내용': '리미트캠스위치교체및조정, 상사점리미트스위치교체'}, {'고장부품': 'QDC 클램프', '불량유형': '기동이상', '조치내용': '에어빼기'}, {'고장부품': '피이더 소재', '불량유형': '진입이상', '조치내용': '캠 스위치 조정'}, {'고장부품': '카운터', '불량유형': '정지시점 불량', '조치내용': '각도 조정'}, {'고장부품': 'NC FEEDER 코일', '불량유형': '이송거리 이상', '조치내용': '재 셋팅'}]


In [ ]:
new_df = pd.DataFrame(result)

In [ ]:
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,None,None,None
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,None,None,None
2,2021-02-20,클램프기동이상 오일보충 에어 제거,None,None,None
3,2021-02-26,피이더 조작판 매입 콘센트 고정,None,None,None
4,2021-04-13,QDC 파손 클램프 교체,None,None,None
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,None,None,None
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,None,None,None
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),None,None,None
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,None,None,None
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,None,None,None


In [ ]:
new_df

,고장부품,불량유형,조치내용
0,F/P,기능상실,PLC 확인 및 클램프 감지센샤 조정
1,와이어,전진동작이상,점검 중 정상 작동됨
2,클램프,기동이상,오일보충 에어 제거
3,피이더 조작판,매입 콘센트,고정
4,QDC,파손,클램프 교체
5,고정블록 볼트,마모,600ton 것으로 대체
6,휠,회전 작동불량,점검 세기정공 AS 요청
7,클러치 ASS'Y,작동이상,분해출문(세기정공)
8,클러치 assy,불량,전자접촉기교체
9,클러치 밋션,기어오일보출,ATF-3


In [ ]:
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,F/P,기능상실,PLC 확인 및 클램프 감지센샤 조정
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,와이어,전진동작이상,점검 중 정상 작동됨
2,2021-02-20,클램프기동이상 오일보충 에어 제거,클램프,기동이상,오일보충 에어 제거
3,2021-02-26,피이더 조작판 매입 콘센트 고정,피이더 조작판,매입 콘센트,고정
4,2021-04-13,QDC 파손 클램프 교체,QDC,파손,클램프 교체
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,고정블록 볼트,마모,600ton 것으로 대체
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,휠,회전 작동불량,점검 세기정공 AS 요청
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),클러치 ASS'Y,작동이상,분해출문(세기정공)
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,클러치 assy,불량,전자접촉기교체
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,클러치 밋션,기어오일보출,ATF-3


In [ ]:
sub_df['고장부품'] = new_df['고장부품']
sub_df['불량유형'] = new_df['불량유형']
sub_df['조치내용'] = new_df['조치내용']

<ipython-input-103-2e3c98d9f99a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['고장부품'] = new_df['고장부품']
<ipython-input-103-2e3c98d9f99a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['불량유형'] = new_df['불량유형']
<ipython-input-103-2e3c98d9f99a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

# 7. ESG 판단 모델 (실제 사례)

모델에게 답을 작성할 때 `근거`를 작성하고나서 결론을 내리라고 하는 방식을 `생각의 사슬(Chain of Thougt) 프롬프팅`이라고 한다.  

이 방식을 사용하면 실제로 모델의 답변의 정확도가 높아질 뿐만 아니라 학습 모델의 판단 근거를 사용자가 확인할 수 있다는 장점이 있다.

In [ ]:
system_prompt = '''당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반이 발생한 경우. 예: 부패 스캔들, 재무 조작
2점: 지배구조 개선 노력이 부족하며, 외부 감사나 투명성 문제에 있는 경우. 예: 의사 결정 과정 불투명, 주주와의 소통 부족
3점: 기본적인 지배구조 원칙을 따르나, 일부 개선의 여지가 있는 경우. 예: 주주와의 소통 강화 중, 투명성 개선 노력
4점: 투명하고 윤리적인 지배구조를 실천하고 있으며, 책임 있는 경영을 하는 경우. 예: 정기적인 외부 감사, 투명한 의사결정 구조
5점: 업계에서 모범이 되는 윤리적 지배구조를 유지하고 지속적 개선을 보이는 경우. 예: 독립적 이사회 구성, 주주 및 이해관계자와의 적극적 소통

## 답변 형식
답변 형식은 반드시 아래의 형식을 따르십시오. 근거:, E:, S:, G: 외에는 그 어떠한 문장도 덧붙이지 마십시오.

근거: 해당 뉴스 기사에 대해서 ESG 점수를 작성한 근거를 작성합니다.
E: 1~5 사이의 정수
S: 1~5 사이의 정수
G: 1~5 사이의 정수

답변 형식 예시는 다음과 같습니다.
근거: 이 뉴스 기사는 방화6구역 조합과 HDC현대산업개발 간의 공사비 갈등과 시공사 계약 해지 과정을 다루고 있습니다. 주 내용은 공사비 인상 문제로 인한 갈등이 심화되어 결국 시공사 교체를 논의하는 상황에 놓여 있다는 것입니다. 이로 인해 사회적 이슈로는 공사 중단 사태가 장기화되고 법적 분쟁까지 이어졌습니다. 환경에 대한 명시적 내용은 없으며, 지배구조와 관련되어 시공사와 조합 간의 협상 실패와 조합원 반발이 드러나고 있습니다.
E: 2
S: 1
G: 2

## 뉴스 기사'''

In [ ]:
user_prompt = "LG家 장녀 ‘주식 부정거래’ 의혹, 금융당국 덜미 잡았다 금감원 [사진 = 연합뉴스]  구연경 LG복지재단 대표가 미공개정보를 이용해 코스닥 상장사 주식을 매수했다는 의혹과 관련해 금융당국이 관련 정황을 포착하고 제재에 착수했다. 구 대표는 고(故) 구본무 LG그룹 선대회장의 장녀다.  27일 금융당국에 따르면 금융감독원은 구 대표가 지난해 코스닥 상장사인 바이오기업 A사 주식을 해당 회사의 투자유치 정보가 알려지기 전에 취득한 것으로 보고 자본시장법 위반 혐의로 해당 안건을 금융위원회에 넘겼다.  금융위는 해당 안건을 내달 2일 열리는 증권선물위원회에 올려 처리할 예정이다. 증선위 결정에 따라 구 대표 관련 사건은 검찰 등 수사기관에 고발 또는 통보될 것으로 보인다.  금감원 고위관계자는 “본인은 혐의를 부인하고 있지만, 거래시점상 미공개정보를 이용했다는 정황증거를 포착했다”며 “의혹이 맞다면 형사처벌 대상이 되는 만큼 검찰 등 수사기관의 수사가 필요한 상황”이라고 설명했다.  앞서 지난해 4월 A사는 BRV(블루런벤처스)캐피탈매니지먼트로부터 제3자 배정 유상증자를 통해 500억원을 조달했다. BRV캐피탈매니지먼트는 실리콘밸리 기반 기관투자사 블루런벤처스의 글로벌 성장 투자 플랫폼이다. 구 대표의 남편인 윤관씨는 BRV캐피탈매니지먼트의 최고투자책임자(CIO)를 맡고 있다.  금융당국은 A사의 자금조달 계획이 알려지기 전 구 대표를 이를 미리 알고 그 전에 본인과 관련인 계좌 등을 이용해 해당 회사 주식 3만주를 개인적으로 취득한 것으로 보고 있다.  만약 구 대표가 주식을 사들인 게 투자 발표 전일 경우, 일반 주주는 알 수 없었던 투자건이라는 미공개 정보를 활용한 자본시장법 위반 행위가 된다.  지난해 3월말 주당 1만6000원 수준이던 A사 주가는 BRV캐피탈머니지먼트의 투자사실이 알려진 당일에만 16% 이상 급등했고, 지난해 한때는 5만원까지 치솟았다.  자본시장법 제174조는 상장법인의 업무 등과 관련한 미공개 중요정보를 특정 증권 등의 매매·거래에 이용하는 행위를 금지하고 있다. 이를 위반하면 1년 이상의 유기징역 또는 그 행위로 얻은 이익 또는 회피한 손실액의 3배 이상 5배 이하에 상당하는 벌금이 매겨진다.  앞서 금감원은 A사의 기타비상무이사(등기임원)를 맡고 있는 B씨를 소환조사한 바 있다. 이후 추가 조사를 통해 구씨의 혐의를 확인한 것으로 보인다.  한편 구 대표는 A사 주식 취득건이 논란이 되자 지난 5월 해당 주식을 LG복지재단에 기부하려고 시도했다. 다만 당시 재단 이사회는 미공개 정보를 활용한 주식 매수가 의심된다는 이유로 관련 안건을 처리하지 않아 실제 기부는 이뤄지지 않았다."

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt + '\n근거:'}
    ]
)

In [ ]:
print(response.choices[0].message.content)

근거: 이번 뉴스 기사는 구연경 LG복지재단 대표의 미공개정보를 이용한 주식 부정거래 의혹과 금융당국의 제재 착수 과정을 다루고 있습니다. 환경에 대한 언급은 없으므로 평가할 사항이 없습니다. 사회적 측면에서는 구 대표가 논란이 된 주식을 기부하려고 시도했으나 실패한 사례가 있으나, 전체적으로 사회적 책임이나 공헌에 대한 내용은 부족합니다. 지배구조 측면에서는 미공개 정보를 이용한 주식 매입 의혹과 같은 불투명한 거래가 문제로 지배구조의 투명성 부족 혹은 부패 사건으로 평가할 수 있습니다.
E: 2
S: 2
G: 1


그렇다면 근거를 작성해달라는 말을 취소하고 요청하면 어떨까?

In [ ]:
system_prompt = '''당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반이 발생한 경우. 예: 부패 스캔들, 재무 조작
2점: 지배구조 개선 노력이 부족하며, 외부 감사나 투명성 문제에 있는 경우. 예: 의사 결정 과정 불투명, 주주와의 소통 부족
3점: 기본적인 지배구조 원칙을 따르나, 일부 개선의 여지가 있는 경우. 예: 주주와의 소통 강화 중, 투명성 개선 노력
4점: 투명하고 윤리적인 지배구조를 실천하고 있으며, 책임 있는 경영을 하는 경우. 예: 정기적인 외부 감사, 투명한 의사결정 구조
5점: 업계에서 모범이 되는 윤리적 지배구조를 유지하고 지속적 개선을 보이는 경우. 예: 독립적 이사회 구성, 주주 및 이해관계자와의 적극적 소통

## 답변 형식
답변 형식은 반드시 아래의 형식을 따르십시오. E:, S:, G: 외에는 그 어떠한 문장도 덧붙이지 마십시오.
E: 1~5 사이의 정수
S: 1~5 사이의 정수
G: 1~5 사이의 정수

답변 형식 예시는 다음과 같습니다.
E: 2
S: 1
G: 2

## 뉴스 기사'''

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt + '\n근거:'}
    ]
)

In [ ]:
print(response.choices[0].message.content)

E: 3
S: 3
G: 1


다른 답을 얻는 것을 볼 수 있다. 다음은 실제로 제작한 데이터를 다운로드하여 확인해보자.



In [ ]:
!wget https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/esg_data_test.json

--2024-12-15 04:04:52--  https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/esg_data_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2806397 (2.7M) [text/plain]
Saving to: ‘esg_data_test.json’

esg_data_test.json  100%[===================>]   2.68M  --.-KB/s    in 0.05s   

2024-12-15 04:04:52 (54.4 MB/s) - ‘esg_data_test.json’ saved [2806397/2806397]



현업에서는 해당 데이터를 바탕으로 LLaMA-3 모델에 학습하여 사용하고 있다.

LLM 파인 튜닝에 관심이 많으신 분들은 ChatGPT API를 이용하여 여러분들의 모델을 학습하기 위한 데이터를 만드는 연습을 많이 해야합니다.